In [1]:
import numpy as np
from cvxopt import matrix, solvers

def svm_qp(X, y):
    m, n = X.shape
    y = y.astype(float)
    
    # Create the P, q, G, h matrices for the QP solver
    K = np.dot(X, X.T) * np.outer(y, y)
    P = matrix(K)
    q = matrix(-np.ones((m, 1)))
    G = matrix(np.diag(-np.ones(m)))
    h = matrix(np.zeros(m))
    A = matrix(y.reshape(1, -1))
    b = matrix(np.zeros(1))

    # Solve QP problem
    sol = solvers.qp(P, q, G, h, A, b)
    alphas = np.array(sol['x']).flatten()
    
    # Compute weight vector w
    w = np.sum(alphas * y[:, None] * X, axis=0)
    
    # Compute bias term b
    support_vectors = (alphas > 1e-5)
    b = np.mean(y[support_vectors] - np.dot(X[support_vectors], w))
    
    return w, b, alphas

# Example usage
X = np.array([[2, 3], [3, 3], [4, 3], [1, 1], [2, 1], [3, 1]])
y = np.array([1, 1, 1, -1, -1, -1])

w, b, alphas = svm_qp(X, y)
print("Weights:", w)
print("Bias:", b)

class SVM:
    def __init__(self, C=1.0, tol=1e-3, max_passes=5):
        self.C = C
        self.tol = tol
        self.max_passes = max_passes

    def fit(self, X, y):
        m, n = X.shape
        self.X = X
        self.y = y
        self.alphas = np.zeros(m)
        self.b = 0
        self.E = np.zeros(m)
        self.K = np.dot(X, X.T)
        
        passes = 0
        while passes < self.max_passes:
            num_changed_alphas = 0
            for i in range(m):
                self.E[i] = self._E(i)
                if (y[i] * self.E[i] < -self.tol and self.alphas[i] < self.C) or \
                   (y[i] * self.E[i] > self.tol and self.alphas[i] > 0):
                    j = np.random.choice([_ for _ in range(m) if _ != i])
                    self.E[j] = self._E(j)
                    
                    alpha_i_old = self.alphas[i].copy()
                    alpha_j_old = self.alphas[j].copy()
                    
                    if y[i] != y[j]:
                        L = max(0, self.alphas[j] - self.alphas[i])
                        H = min(self.C, self.C + self.alphas[j] - self.alphas[i])
                    else:
                        L = max(0, self.alphas[j] + self.alphas[i] - self.C)
                        H = min(self.C, self.alphas[j] + self.alphas[i])
                        
                    if L == H:
                        continue
                        
                    eta = 2 * self.K[i, j] - self.K[i, i] - self.K[j, j]
                    if eta >= 0:
                        continue
                    
                    self.alphas[j] -= y[j] * (self.E[i] - self.E[j]) / eta
                    self.alphas[j] = np.clip(self.alphas[j], L, H)
                    
                    if abs(self.alphas[j] - alpha_j_old) < 1e-5:
                        continue
                        
                    self.alphas[i] += y[i] * y[j] * (alpha_j_old - self.alphas[j])
                    
                    b1 = self.b - self.E[i] - y[i] * (self.alphas[i] - alpha_i_old) * self.K[i, i] - \
                         y[j] * (self.alphas[j] - alpha_j_old) * self.K[i, j]
                    b2 = self.b - self.E[j] - y[i] * (self.alphas[i] - alpha_i_old) * self.K[i, j] - \
                         y[j] * (self.alphas[j] - alpha_j_old) * self.K[j, j]
                    
                    if 0 < self.alphas[i] < self.C:
                        self.b = b1
                    elif 0 < self.alphas[j] < self.C:
                        self.b = b2
                    else:
                        self.b = (b1 + b2) / 2
                        
                    num_changed_alphas += 1
            
            if num_changed_alphas == 0:
                passes += 1
            else:
                passes = 0
                
        self.w = np.dot((self.alphas * y).T, X)
        
    def _E(self, i):
        return self.b + np.dot(self.alphas * self.y, self.K[:, i]) - self.y[i]
    
    def predict(self, X):
        return np.sign(np.dot(X, self.w) + self.b)

# Example usage
X = np.array([[2, 3], [3, 3], [4, 3], [1, 1], [2, 1], [3, 1]])
y = np.array([1, 1, 1, -1, -1, -1])

svm = SVM(C=1.0)
svm.fit(X, y)
predictions = svm.predict(X)
print("Predictions:", predictions)


# Assuming X_test is your test data
X_test = np.array([[2, 2], [3, 2], [4, 2], [1, 0], [2, 0], [3, 0]])
predictions = svm.predict(X_test)
print("Test Predictions:", predictions)


     pcost       dcost       gap    pres   dres
 0: -4.8077e-01 -1.3432e+00  9e-01  2e-16  1e+00
 1: -4.9687e-01 -5.3098e-01  3e-02  6e-17  1e-01
 2: -5.0000e-01 -5.0043e-01  4e-04  6e-17  4e-04
 3: -5.0000e-01 -5.0000e-01  4e-06  8e-17  4e-06
 4: -5.0000e-01 -5.0000e-01  4e-08  7e-17  4e-08
Optimal solution found.


ValueError: operands could not be broadcast together with shapes (6,6) (6,2) 